In [46]:
from __future__ import absolute_import, division, print_function, unicode_literals
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [47]:
#데이터를 불러와

import numpy as np

rows = np.loadtxt("./lotto.csv", delimiter=",")
row_count = len(rows)
print(row_count)

985


In [48]:
import numpy as np

# 당첨번호를 원핫인코딩벡터(ohbin)으로 변환
def numbers2ohbin(numbers):

    ohbin = np.zeros(45) #45개의 빈 칸을 만듬

    for i in range(6): #여섯개의 당첨번호에 대해서 반복함
        ohbin[int(numbers[i])-1] = 1 #로또번호가 1부터 시작하지만 벡터의 인덱스 시작은 0부터 시작하므로 1을 뺌
    
    return ohbin

# 원핫인코딩벡터(ohbin)를 번호로 변환
def ohbin2numbers(ohbin):

    numbers = []
    
    for i in range(len(ohbin)):
        if ohbin[i] == 1.0: # 1.0으로 설정되어 있으면 해당 번호를 반환값에 추가한다.
            numbers.append(i+1)
    
    return numbers

In [49]:
numbers = rows[:, 1:7]
ohbins = list(map(numbers2ohbin, numbers))

x_samples = ohbins[0:row_count-1]
y_samples = ohbins[1:row_count]

#원핫인코딩으로 표시
print("ohbins")
print("X[0]: " + str(x_samples[0]))
print("Y[0]: " + str(y_samples[0]))

#번호로 표시
print("numbers")
print("X[0]: " + str(ohbin2numbers(x_samples[0])))
print("Y[0]: " + str(ohbin2numbers(y_samples[0])))

ohbins
X[0]: [0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0.
 0. 0. 0. 0. 1. 0. 0. 0. 1. 0. 0. 0. 1. 0. 0. 1. 0. 0. 0. 0. 0.]
Y[0]: [0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0.
 1. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0.]
numbers
X[0]: [10, 23, 29, 33, 37, 40]
Y[0]: [9, 13, 21, 25, 32, 42]


In [50]:
#windows를 만들자

seq_len = 50
sequence_length = seq_len + 1

result=[]
for index in range(len(x_samples)-sequence_length):
    result.append(x_samples[index: index+sequence_length])

    

In [51]:
result = np.array(x_samples)

row = int(round(result.shape[0]*0.9))
train = result[:row, :]
np.random.shuffle(train)

x_train = train[:, :-1]
x_train = np.reshape(x_train,(x_train.shape[0], x_train.shape[1],-1))
y_train = train[:,-1]

x_test = result[row:, :-1]
x_test = np.reshape(x_test,(x_test.shape[0], x_test.shape[1],-1))
y_test = result[row:,-1]

x_train.shape, x_test.shape


((886, 44, 1), (98, 44, 1))

In [52]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras import models
from tensorflow.keras.losses import Huber
from tensorflow.keras.optimizers import Adam

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, LSTM, Conv1D, Lambda

#모델을 컴파일 해보자
model = Sequential()
model.add(LSTM(50,return_sequences=True,input_shape=(50,1)))
model.add(LSTM(64, return_sequences=False))
model.add(Dense(45, activation='relu'))
model.compile(loss='binary_crossentropy', optimizer='adam')
model.summary()


Model: "sequential_5"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_10 (LSTM)               (None, 50, 50)            10400     
_________________________________________________________________
lstm_11 (LSTM)               (None, 64)                29440     
_________________________________________________________________
dense_4 (Dense)              (None, 45)                2925      
Total params: 42,765
Trainable params: 42,765
Non-trainable params: 0
_________________________________________________________________


In [53]:
#Training

model.fit(x_train, y_train,
         validation_data=(x_test, y_test),
         batch_size=10,
         epochs=50)

Epoch 1/50
89/89 [==============================] - 9s 53ms/step - loss: 1.0303 - val_loss: 0.6604
Epoch 2/50
89/89 [==============================] - 4s 43ms/step - loss: 0.9796 - val_loss: 0.6650
Epoch 3/50
89/89 [==============================] - 5s 52ms/step - loss: 0.9788 - val_loss: 0.6553
Epoch 4/50
89/89 [==============================] - 5s 60ms/step - loss: 0.9752 - val_loss: 0.6590
Epoch 5/50
89/89 [==============================] - 4s 44ms/step - loss: 0.9749 - val_loss: 0.6628
Epoch 6/50
89/89 [==============================] - 4s 43ms/step - loss: 0.9244 - val_loss: 0.5967
Epoch 7/50
89/89 [==============================] - 4s 43ms/step - loss: 0.8561 - val_loss: 0.5900
Epoch 8/50
89/89 [==============================] - 4s 44ms/step - loss: 0.8530 - val_loss: 0.6070
Epoch 9/50
89/89 [==============================] - 4s 43ms/step - loss: 0.8552 - val_loss: 0.5836
Epoch 10/50
89/89 [==============================] - 4s 43ms/step - loss: 0.8506 - val_loss: 0.5922
Epoch 11/

In [54]:
def gen_numbers_from_probability(nums_prob):

    ball_box = []

    for n in range(45):
        ball_count = int(nums_prob[n] * 100 + 1)
        ball = np.full((ball_count), n+1) #1부터 시작
        ball_box += list(ball)

    selected_balls = []

    while True:
        
        if len(selected_balls) == 6:
            break
        
        ball_index = np.random.randint(len(ball_box), size=1)[0]
        ball = ball_box[ball_index]

        if ball not in selected_balls:
            selected_balls.append(ball)

    return selected_balls

In [59]:
# 마지막 회차까지 학습한 모델로 다음 회차 추론

print('receive numbers')

xs = np.reshape(x_test,(x_test.shape[0], x_test.shape[1],-1))

ys_pred = model.predict_on_batch(xs)

list_numbers = []

for n in range(10):
    numbers = gen_numbers_from_probability(ys_pred[0])
    numbers.sort()
    print('{0} : {1}'.format(n, numbers))
    list_numbers.append(numbers)
    

receive numbers
0 : [11, 16, 23, 26, 35, 42]
1 : [1, 18, 22, 29, 33, 36]
2 : [19, 21, 26, 31, 32, 43]
3 : [5, 7, 13, 28, 41, 43]
4 : [2, 3, 7, 8, 20, 27]
5 : [4, 11, 17, 20, 29, 34]
6 : [1, 3, 22, 28, 33, 43]
7 : [8, 10, 14, 27, 31, 35]
8 : [7, 11, 22, 34, 41, 42]
9 : [10, 19, 23, 31, 37, 43]
